In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
import pandas as pd
from matplotlib import rcParams

rcParams['font.sans-serif'] = ['SimHei']
rcParams['axes.unicode_minus'] = False

components = {
    '零配件1': {'次品率': 0.1, '购买单价': 2, '检测成本': 1},
    '零配件2': {'次品率': 0.1, '购买单价': 8, '检测成本': 1},
    '零配件3': {'次品率': 0.1, '购买单价': 12, '检测成本': 2},
    '零配件4': {'次品率': 0.1, '购买单价': 2, '检测成本': 1},
    '零配件5': {'次品率': 0.1, '购买单价': 8, '检测成本': 1},
    '零配件6': {'次品率': 0.1, '购买单价': 12, '检测成本': 2},
    '零配件7': {'次品率': 0.1, '购买单价': 8, '检测成本': 1},
    '零配件8': {'次品率': 0.1, '购买单价': 12, '检测成本': 2},
}

semi_products = {
    '半成品1': {'次品率': 0.1, '装配成本': 8, '检测成本': 4, '拆解费用': 6},
    '半成品2': {'次品率': 0.1, '装配成本': 8, '检测成本': 4, '拆解费用': 6},
    '半成品3': {'次品率': 0.1, '装配成本': 8, '检测成本': 4, '拆解费用': 6},
}

final_product = {
    '成品': {'次品率': 0.1, '装配成本': 8, '检测成本': 6, '拆解费用': 10, '售价': 200, '调换损失': 40}
}

def calculate_expected_cost(components, semi_products, final_product, detect_components, detect_semi, detect_final,
                            dismantle):
    total_cost = 0
    total_defective_rate = 1

    for i, (comp_name, comp_data) in enumerate(components.items()):
        cost = comp_data['购买单价']
        defective_rate = comp_data['次品率']
        if detect_components[i]:
            defective_rate *= 0.5
            cost += comp_data['检测成本']

        total_cost += cost
        total_defective_rate *= (1 - defective_rate)

    for i, (semi_name, semi_data) in enumerate(semi_products.items()):
        cost = semi_data['装配成本']
        defective_rate = semi_data['次品率']
        if detect_semi[i]:
            defective_rate *= 0.5
            cost += semi_data['检测成本']

        total_cost += cost
        total_defective_rate *= (1 - defective_rate)

    final_defective_rate = final_product['成品']['次品率'] * (0.5 if detect_final else 1)
    final_cost = final_product['成品']['装配成本']

    if detect_final:
        final_cost += final_product['成品']['检测成本']

    total_cost += final_cost
    total_defective_rate = 1 - (total_defective_rate * (1 - final_defective_rate))

    exchange_loss = final_product['成品']['调换损失'] * total_defective_rate
    total_cost += exchange_loss

    if dismantle:
        total_cost += final_product['成品']['拆解费用']

    return total_cost, total_defective_rate

def generate_strategy_description(detect_components, detect_semi, detect_final, dismantle):
    description = ""
    for i, detect in enumerate(detect_components):
        description += f"检测零配件 {i + 1}，" if detect else f"不检测零配件 {i + 1}，"
    for i, detect in enumerate(detect_semi):
        description += f"检测半成品 {i + 1}，" if detect else f"不检测半成品 {i + 1}，"
    if detect_final:
        description += "检测成品，"
    else:
        description += "不检测成品，"
    if dismantle:
        description += "拆解不合格成品"
    else:
        description += "不拆解不合格成品"
    return description

def fitness_function(chromosome, components, semi_products, final_product):
    detect_components = chromosome[:8]
    detect_semi = chromosome[8:11]
    detect_final = chromosome[11]
    dismantle = chromosome[12]

    cost, defective_rate = calculate_expected_cost(components, semi_products, final_product, detect_components,
                                                   detect_semi, detect_final, dismantle)
    return cost, defective_rate

def initialize_population(population_size, num_genes):
    return np.random.randint(2, size=(population_size, num_genes))

def selection(population, fitness_values, num_parents):
    parents = np.empty((num_parents, population.shape[1]), dtype=int)
    for i in range(num_parents):
        min_fitness_idx = np.argmin(fitness_values)
        parents[i, :] = population[min_fitness_idx, :]
        fitness_values[min_fitness_idx] = np.inf
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size, dtype=int)
    crossover_point = np.uint8(offspring_size[1] / 2)

    for k in range(offspring_size[0]):
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

def mutation(offspring_crossover, mutation_rate):
    for idx in range(offspring_crossover.shape[0]):
        if np.random.rand() < mutation_rate:
            random_gene_idx = np.random.randint(offspring_crossover.shape[1])
            offspring_crossover[idx, random_gene_idx] = 1 - offspring_crossover[idx, random_gene_idx]
    return offspring_crossover

def genetic_algorithm(components, semi_products, final_product, population_size=10000, num_generations=100, mutation_rate=0.01):
    num_genes = 13
    population = initialize_population(population_size, num_genes)
    results = []

    for generation in range(num_generations):
        fitness_values = np.array([fitness_function(chromosome, components, semi_products, final_product)[0] for chromosome in population])
        parents = selection(population, fitness_values, population_size // 2)
        offspring_crossover = crossover(parents, offspring_size=(population_size - parents.shape[0], num_genes))
        offspring_mutation = mutation(offspring_crossover, mutation_rate)
        population[0:parents.shape[0], :] = parents
        population[parents.shape[0]:, :] = offspring_mutation

        # 记录每一代的最优策略
        best_chromosome = population[np.argmin(fitness_values)]
        best_cost, best_defective_rate = fitness_function(best_chromosome, components, semi_products, final_product)
        best_strategy_description = generate_strategy_description(best_chromosome[:8], best_chromosome[8:11], best_chromosome[11], best_chromosome[12])
        results.append([generation + 1, best_strategy_description, best_cost, best_defective_rate])

    # 找到最优策略
    best_chromosome = population[np.argmin(fitness_values)]
    best_cost, best_defective_rate = fitness_function(best_chromosome, components, semi_products, final_product)
    best_strategy_description = generate_strategy_description(best_chromosome[:8], best_chromosome[8:11], best_chromosome[11], best_chromosome[12])

    return best_strategy_description, best_cost, best_defective_rate, results

# 使用遗传算法求解最优策略
best_strategy_description, best_cost, best_defective_rate, results = genetic_algorithm(components, semi_products, final_product)

# 将结果保存到Excel文件中
df = pd.DataFrame(results, columns=['策略编号', '策略描述', '总成本', '成品次品率'])
df.to_excel('策略结果.xlsx', index=False)
print("所有策略已经保存到 '策略结果.xlsx' 文件中。")

print(f"最优策略: {best_strategy_description}")
print(f"最优策略总成本: {best_cost:.2f}")
print(f"最优策略成品次品率: {best_defective_rate:.2f}")

# 可视化结果
plt.figure(figsize=(10, 6))
plt.plot(df['策略编号'], df['总成本'], marker='o', linestyle='-', color='blue', label="总成本")
plt.xlabel("策略编号")
plt.ylabel("总成本")
plt.title("不同策略下的总成本比较")
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(df['策略编号'], df['成品次品率'], marker='x', linestyle='--', color='red', label="成品次品率")
plt.xlabel("策略编号")
plt.ylabel("成品次品率")
plt.title("不同策略下的成品次品率比较")
plt.grid(True)
plt.legend()
plt.show()